<h1 style="color:black" align="center">Решающие деревья</h1>

<h1 style="color:#008B8B">1. Определение решающего дерева</h1>

Рассмотрим бинарное дерево, в котором:

* каждой внутренней вершине $v$ приписана функция (или предикат) $\beta_v: \mathbb{X} \to \{0, 1\}$;

* каждой листовой вершине $v$ приписан прогноз $c_v \in Y$ (в случае с классификацией листу также может быть приписан вектор вероятностей).


Рассмотрим теперь алгоритм $a(x)$, который стартует из корневой вершины $v_0$ и вычисляет значение функции $\beta_{v_0}$. Если оно равно нулю, то алгоритм переходит в левую дочернюю вершину, иначе в правую, вычисляет значение предиката в новой вершине и делает переход или влево, или вправо. Процесс продолжается, пока не будет достигнута листовая вершина; алгоритм возвращает тот класс, который приписан этой вершине. Такой алгоритм называется - *бинарным решающим деревом*.

На практике в большинстве случаев используются одномерные предикаты $\beta_v$, которые сравнивают значение одного из признаков с порогом:

$\beta_v(x; j, t) = [x_j < t].$

Существуют и многомерные предикаты, например:

* линейные $\beta_v(x) = [\langle w, x \rangle < t]$;

* метрические $\beta_v(x) = [\rho(x, x_v) < t]$, где точка $x_v$ является одним из объектов выборки любой точкой признакового пространства. $\rho(x, x_v)$ - расстояние от объекта $x$ до объекта $x_v$.

Многомерные предикаты позволяют строить ещё более сложные разделяющие поверхности, но очень редко используются на практике - например, из-за того, что усиливают и без того выдающиеся способности деревьев к переобучению. Далее мы будем говорить только об одномерных предикатах.

<h1 style="color:#008B8B">2. Построение деревьев</h1>

Допутсим, у нас имеется два признака. Визуализируем точки на графике, по оси $x$ обозначим признак $x_1$, по оси $y$ обозначим признак $x_2$. Тогда при построении бинарного дерева, мы начнем разделять плоскость полосами на небольшие части. Можно показать, что если нет двух объектов с одинаковыми признаками, но разными классами, тогда можно получить нулевую ошибку на обучающей выборке. Линейные модели являются куда более простыми, ибо они не способы переобучиться настолько сильно.

Пример переобучения, если имеются два класса (крестик, нолик) и в нутри крестиков есть один нолик, тогда решающее дерево постарается выделить участок для нолика среди всех крестиков. Естественно, для тестовой выборки в этой выделенной области для нолика - должны лежать крестики, а модель скажет что там нолики, что снизит качество.

Если у дерева много узлов, много листьев, тогда скорее всего оно будет переобученным. Мы хотим чтобы алгоритм построения дерева находил дерево с минимальным числом вершин и наименьшей ошибкой - эта задача является NP-полная.

### Жадное построение дерва

Опишем базовый жадный алгоритм построения бинарного решающего дерева. Запишем алгоримт в виде псевдокода:

Имеется функция $\text{BulidNode}(m, R_m):$, которая на вход принимает номер вершины $m$ и объекты которые попали в данную вершину $R_m$. Функция будет запускаться с аргументами $\text{BulidNode}(1, X)$, где $X$ - это вся обучающая выборка.

**Шаг 1.** Проверяем, необходимо ли продолжать построение дерева. Есил выполнен критерий остановки: 

* тогда считаем прогноз $c_m$ и завершаем работу объявив вершину листовой.

**Шаг 2.** Иначе, необходимо продолжить разбиение. Тогда необходимо найти оптимальный предикат, в качестве предиката мы будем использовать $[x_j < t]$, значит необходимо найти оптимальные $j, t$ - запишем это как:

$$\large j, t = \underset{\substack{j = 1, \ldots, d \\t}}{\text{arg max}} \quad Q(R_m, j, t)$$ - Выполняем $\text{arg max}$ по всем признакам $j$ и по всем порогам $t$, считаем значение функционала качества предиката. Другими словами мы выполняем полный перебор всех признаков, всех порогов и берем ту пару, на которой значение функционала качества для предиката максимальное.
    
Значит, дальше необходимо выполнить разбиение: Множество объектов, которые пойдут влево $R_{\ell} = \{(x, y) \in R_{m} | [x_{j} \le t]\}$, а вправо $R_{r} = \{(x, y) \in R_{m} | [x_{j} > t]\}$

**Шаг 3.** Запускаем функцию  для левого поддерева $\text{BulidNode}(l, R_{\ell})$ и правого поддерева $\text{BulidNode}(r, R_{r})$
    

**Критерии остановки:**

* Для задачи классификации  - если среди всех объектов, которые попали в вершину $R_m$, содержатся объекты только одного класса, тогда можно остановиться.

* Если достигнута максимальная глубина

* Если мощность $|R_m| < K$ меньше $K$, то есть, если в $R_m$ содержится мало объектов, тогда нет смысла продолжать разбиение. 


**Как посчитать прогноз $c_m$**

Если мы вершину $c_m$ объявляем листовой, дальше не разбиваем, так как это лист, тогда необходимо выдавать некоторый прогноз на листе:

* Для задачи классификации - Выдаем самый популярный класс в $R_m$

* Для задачи классификации - Выдаем распределение классов в $R_m$

* Для задачи регрессии - Выдаем среднее значение по $R_m$

**Пороги**

Берем $j$-й признак и сортируем все объекты по значению данного признака $x_{1 j} \le \ldots \le x_{\ell j}$. Тогда берем все промежуточные пороги (между каждой парой). Но не понятно какое значение порога брать, можем вспомнить идею о SVM, что нет необходимости делать лишних предположения о данных, разделяя выборки на пополам. Значит мы возьмем все попроги между значениями признаков $t_{1} = \frac{x_{1 j} + x_{2 j}}{2}, \ldots$. 

Из-за большого количества объектов, количество порогов будет огромным и перебор для разбиеня усложниться, так как мы находимо оптимальную пару из $j, t$. Соответственно, для отсортированных значений признаков $x_{1 j} \le \ldots \le x_{\ell j}$ и использовать в качестве порога перцентили, например 100 штук.

<h1 style="color:#008B8B">3. Критерии информативности</h1>

Введем функцию $H(R)$, которая считается конкретно для вершины дерева - это критерий информативности / мера хаотичности вершины (impurity criterion). Это функция показыват насколько неоднородно распределение целевой переменной в этой верщине.

Усрединм по всем объектам, которые попали в множество $R$ и посчитаем ошибку при прогнозировании некоторой константой $c$ и возьмем минимум по всем таким констатам $c$:

$$\large H(R) = \min_{c \in \mathbb{Y}} \frac{1}{|R|} \sum_{(x_i, y_i) \in R} L(y_i, c)$$

Представляем, что эта вершина $R$ будет являться листой и дальше мы ее не будем разбивать, тогда мы должны на этой вершине выдавать константный прогноз. Тогда мы перебираем все возможные константные прогнозы в этой вершине и для каждого констангного прогноза посмотрим насколько он ошибается, какова ошибка константного прогноза на выборке $R$ и выберем такой константный прогноз при котором ошибка будет минимальна и это будет значением impurity.

Если мы решаем задачу классификации и в нашей вершине три положительных объекта, тогда если взять константу $c = +1$, тогда значение функции потерь будет нулевое и impurity тоже будет равен $H(R) = 0$. А если вершина будет разнородная тогда для любой константы значение $H(R)$ окажется больше.

<h2 style="color:#008B8B">3.1 Регрессия</h2>

Как обычно, в регрессии выберем квадрат отклонения в качестве функции потерь $L(y, c) = (y - c)^2$. В этом случае критерий информативности будет выглядеть как:

$$\large H(R) = \min_{c \in \mathbb{Y}} \frac{1}{|R|} \sum_{(x_i, y_i) \in R} (y_i - c)^2$$

Как известно, минимум в этом выражении будет достигаться на среднем значении целевой переменной. Мы значем, что в такой функции оптимальны прогноз $c$ это среднее значение $y$ по всей выборке $R$, запишем как $c = \frac{1}{|R|} \sum\limits_{(x_i, y_i) \in R} y_i = \bar{y}$, тогда $H(R)$ считается как:

$$\large H(R) = \frac{1}{|R|} \sum_{(x_i, y_i) \in R} (y_i - \bar{y})^2$$

Мы получили, что информативность вершины измеряется её дисперсией — чем ниже разброс целевой переменной, тем лучше вершина. Разумеется, можно использовать и другие функции ошибки L — например, при выборе абсолютного отклонения мы получим в качестве критерия среднее абсолютное отклонение от медианы.

Если в вершине находится три объекта с одним значением целевой переменной, тогда дисперсия в этой выборке будет равна 0.

<h2 style="color:#008B8B">3.2 Классификация</h2>

Обозначим через $p_{k}$ долю объектов класса $k$ ($k \in \{1, \dots, K\}$), попавших в вершину $R$:

$p_{k} = \frac{1}{|R|} \sum\limits_{(x_i, y_i) \in R} [y_i = k].$

Через $k_*$ обозначим класс, чьих представителей оказалось больше всего среди объектов, попавших в данную вершину: $k_* = \underset{\substack{k}}{\text{arg max}} p_{k}$.

### 3.2.1 Ошибка классификации

Рассмотрим индикатор ошибки как функцию потерь $L(y, c) = [y \ne c]$:

$$\large H(R) = \min_{c \in \mathbb{Y}} \frac{1}{|R|} \sum_{(x_i, y_i) \in R} [y \ne c]$$

Легко видеть, что оптимальным предсказанием тут будет наиболее популярный класс $k_{*}$ — значит, критерий будет равен следующей доле ошибок:

$$\large H(R) = \min_{c \in \mathbb{Y}} \frac{1}{|R|} \sum_{(x_i, y_i) \in R} [y \ne k_{*}] = 1 - p_{k_{*}}$$

Данный критерий является достаточно грубым, поскольку учитывает частоту $p_{k_{*}}$ лишь одного класса. Например, у вершыны находится 3 крестика и 3 нолика, тогда мы получим $H(R) = \frac{1}{2}$. В лучшем случае можно добавить ещё один предикат и провести разбиение, после чего можно будет остановиться. А если будет 3 крестика и ещё 3 объекта других классов, тогда $H(R) = \frac{1}{2}$, когда он должен быть ниже. Но для нашей модели первый вариант был лучше, так как классов много и за одно разбиение будет сложно разделить классы.

### 3.2.2 Критерий Джини

Рассмотрим ситуацию, в которой мы выдаём в вершине не один класс,
а распределение на всех классах $c = (c_1, \dots, c_K)$ или же вектор вероятностей, где $\sum\limits_{k = 1}^{K} c_k = 1$ и $c_k \ge 0$. Качество такого распределения можно измерять, например, с помощью критерия Бриера (Brier score):

$\large L(y, c) = \sum\limits_{k=1}^K (c_k - [y = k])^2$ - берем веротяность каждого класса и требуем, чтобы для правильного класса вероятность была как можно ближе к 1, а для всех отсальных классов она была как можно ближе к 0. Можно показать, что эта фукнция потреь удовлетворяет требованию корректного оценивания вероятностей.

**Запишем impurity:**

$$\large H(R)
=
\min_{\substack{\sum_k c_k = 1 \\(c_1, \dots, c_K)}}
\frac{1}{|R|}
\sum_{(x_i, y_i) \in R}
\sum_{k = 1}^{K}
    (c_k - [y_i = k])^2.$$
    
    
$\min_{\substack{\sum_k c_k = 1 \\(c_1, \dots, c_K)}}$ - минимум по всем векторам с еденичной суммой.

Имеется вектор вероятностей классов $c = (c_1, \dots, c_K)$. Имеется $K$ классов и выдаем прогноз в виде вектора вероятностей этих классов. Для вершины выдаем один вектор вероятностей классов и считаем качество этих вероятностей. Для этого мы суммируем по всем объектам и для каждого объекта смотрим насколько вероятности калссов совпадают с классом объекта. Мы хотим чтобы для положительных объектов вероятность правильного класса была ближе к 1, а для отрицательных вероятность ближе к 0.

Например, если у нас внутри вершина находится 3 класса, тогда мы выдаем вектор вероятности мощностью 3. И мы хотим узнать то, насколько вероятности подходят для набора объектов внутри вершины. И по формуле выше измеряем качество оценивания вероятностей по этому вектору. 

**Оптимальное $c_k$**:

Перепишем формулу:

$$\large H(R)
=
\min_{\substack{\sum_k c_k = 1 \\(c_1, \dots, c_K)}}
\sum_{k = 1}^{K}
\frac{1}{|R|}
\sum_{(x_i, y_i) \in R} 
    (c_k - [y_i = k])^2$$
    
Так как $c_k$, входит только в вторую сумму, тогда проминимизируем каждое слогаемое в отдельности, независимо. Но при минимизации имеется ограничение $\sum_k c_k = 1$, что сумма должна равняться еденицы. Давайте забудем про это условие и проминимизируем каждое слогаемое в отдельности, независимо:

$$\large = \frac{1}{|R|} \sum_{k = 1}^{K} \underset{c_k \in R}{\text{min}} \sum_{(x_i, y_i) \in R} (c_k - [y_i = k])^2$$

Следовательно, внутри $k$-го слогаемого (вторая сумма) $c_k$ это константа, для которой необходимо найти оптимальное значение. Чему равняется оптимальное значение $c_k$ для суммы квадратов отклонения от некоторой выборки? Тогда оптимальное $c_k$:

$c_k = \frac{1}{|R|} \sum\limits_{(x_i, y_i) \in R} [y_i = k]$, 

А это и есть обозначение, которе мы ввели выше

$p_{k} = \frac{1}{|R|} \sum\limits_{(x_i, y_i) \in R} [y_i = k]$

**Проверка условия $\sum\limits_k c_k = 1$**

Мы опустили условие того, что $\sum_k c_k = 1$, давайте проверим, выполняется ли оно:

$\sum\limits_{k=1}^K c_k = \frac{1}{|R|} \sum\limits_{k=1}^{K} \sum\limits_{(x_i, y_i) \in R} [y_i = k] =$

$= \frac{1}{|R|} \sum\limits_{(x_i, y_i) \in R} \sum\limits_{k=1}^{K} [y_i = k]$

Получаем, что последняя сумма всегда равна 1, так как каждый объект относится ровно к одному классу. Ну и сумму из таких едениц мощностью $|R|$ и все умножается на $\frac{1}{|R|}$, следовательно, получаем, что $c_k$ суммируются в 1.


**Тогда, impurity можно записать как:**

$$\large H(R)
=
\frac{1}{|R|}
\sum_{(x_i, y_i) \in R}
\sum_{k = 1}^{K}
    (p_k - [y_i = k])^2 = $$
    
Расскроем скобки:

$$\large = \frac{1}{|R|}
\sum_{(x_i, y_i) \in R}
\sum_{k = 1}^{K}
    (p_k^2 - 2\cdot [y_i = k] p_k + [y_i = k]) =$$


**Слогаеомое 1** Запишем первое слогаемое отдельно и поменяем суммы местами:

$\frac{1}{|R|} \sum\limits_{k = 1}^{K} \sum\limits_{(x_i, y_i) \in R} p_k^2 = $

$p_k$ не зависит от второй суммы, так $p_k$ нет зависимости от $(x_i, y_i)$, тогда, заменим сумму на $|R|$ и можем сократить на $|R|$:

$= \frac{1}{|R|} \sum\limits_{k = 1}^{K} |R| \cdot p_k^2 = \sum\limits_{k = 1}^{K} p_k^2$


**Слогаеомое 2** Запишем второе слогаемое отдельно и поменяем суммы местами, где $p_k$ можно вынести за скобку, так как оно не зависит от суммы

$- \frac{2}{|R|} \sum\limits_{k = 1}^{K} p_k \sum\limits_{(x_i, y_i) \in R} [y_i = k] = $

Запишем в следующем виде:

$= - 2 \sum\limits_{k = 1}^{K} p_k \frac{1}{|R|} \sum\limits_{(x_i, y_i) \in R} [y_i = k] = $

Где последняя сумма - это $p_k$:

$= - 2 \sum\limits_{k = 1}^{K} p_k^2 $


**Слогаеомое 3**

Запишем третье слогаемое отдельно и поменяем суммы местами:

$\sum\limits_{k = 1}^{K} \frac{1}{|R|} \sum\limits_{(x_i, y_i) \in R} [y_i = k] =$

А это и есть:

$= \sum\limits_{k = 1}^{K} p_k$


**Запишем все вместе:***

$$\large \sum\limits_{k = 1}^{K} p_k^2 - 2 \sum\limits_{k = 1}^{K} p_k^2 + \sum\limits_{k = 1}^{K} p_k = $$

Сокращаем и получаем

$$= \large \sum\limits_{k = 1}^{K} (p_k - p_k^2) = \sum\limits_{k = 1}^{K} p_k(1- p_k)$$



Можно показать, что оптимальный вектор вероятностей состоит из долей классов $p_k$:

$$c_* = (p_1, \dots, p_K)$$


Если подставить эти вероятности в исходный критерий информативности
и провести ряд преобразований, то мы получим критерий Джини: 

$$\large H(R) = \sum_{k = 1}^{K} p_k (1 - p_k).$$

Ну и можно проверить, что у этого критерия значение будет равно нулю если класс один в вершине и его значение будет максиально, если распределение по классом равномерное.


**Оптимальное $c_k$**:

Перепишем формулу:

$$\large H(R)
=
\min_{\substack{\sum_k c_k = 1 \\(c_1, \dots, c_K)}}
\sum_{k = 1}^{K}
\frac{1}{|R|}
\sum_{(x_i, y_i) \in R} 
    (c_k - [y_i = k])^2$$

Следовательно, внутри $k$-го слогаемого (вторая сумма) $c_k$ это константа, для которой необходимо найти оптимальное значение. Чему равняется оптимальное значение $c_k$ для суммы квадратов отклонения от некоторой выборки? Тогда оптимальное $c_k$: 

$c_k = \frac{1}{|R|} \sum\limits_{(x_i, y_i) \in R} [y_i = k]$, 



### 3.2.3 Энтропийный критерий

Мы уже знакомы с более популярным способом оценивания качества вероятностей - логарифмическими потерями, или логарифмом правдоподобия:

$$\large H(R)
=
\min_{\sum\limits_k c_k = 1} \left(
    -
    \frac{1}{|R|}
    \sum\limits_{(x_i, y_i) \in R}
    \sum\limits_{k = 1}^{K}
        [y_i = k]
        \log c_k
\right)$$

Для вывода оптимальных значений $c_k$ вспомним, что все значения $c_k$
должны суммироваться в единицу. Как известного из методов оптимизации, для учёта этого ограничения необходимо искать минимум лагранжиана:

$$\large L(c, \lambda)
=
-
\frac{1}{|R|}
\sum\limits_{(x_i, y_i) \in R}
\sum\limits_{k = 1}^{K}
    [y_i = k]
    \log c_k
+
\lambda
\sum\limits_{k = 1}^{K}
    c_k
\to
\min_{c_k}$$

Дифференцируя, получаем:

$$\frac{\partial}{\partial c_k}
L(c, \lambda)
=
-
\frac{1}{|R|}
\sum\limits_{(x_i, y_i) \in R}
    [y_i = k]
    \frac{1}{c_k}
+
\lambda
=
- \frac{p_k}{c_k}
+
\lambda
=
0,$$

откуда выражаем $c_k = p_k / \lambda$. Суммируя эти равенства по $k$, получим

$$\large 1 = \sum\limits_{k = 1}^{K} c_k = \frac{1}{\lambda} \sum\limits_{k = 1}^{K} p_k = \frac{1}{\lambda},$$

откуда $\lambda = 1$. Значит, минимум достигается при $c_k = p_k$, как и в предыдущем случае. Подставляя эти выражения в критерий, получим, что он будет представлять собой энтропию распределения классов:

$$\large H(R) = - \sum\limits_{k = 1}^{K} p_k \log p_k.$$

Из теории вероятностей известно, что энтропия ограничена снизу нулем, причем минимум достигается на вырожденных распределениях ($p_i = 1$, $p_j = 0$ для $i \neq j$). Максимальное же значение энтропия принимает для равномерного распределения. Отсюда видно, что энтропийный критерий отдает предпочтение более "вырожденным" распределениям классов
в вершине.

### Выбор лучшего предиката

При построении дерева необходимо задать *функционал качества*, на основе которого осуществляется разбиение выборки на каждом шаге. Обозначим через $R_m$ множество объектов, попавших в вершину, разбиваемую на данном шаге, а через $R_\ell$ и $R_r$ - объекты, попадающие в левое и правое поддерево соответственно при заданном предикате.

Мы будем использовать функционалы следующего вида:

$$\large Q(R_m, j, s)
=
H(R_m)
-
\frac{|R_\ell|}{|R_m|}
H(R_\ell)
-
\frac{|R_r|}{|R_m|}
H(R_r).$$

**Пояснение функционала**

Считаем хаотичность до разбиения и считаем хаотичность для каждой из вершин после разбиения, и чем больше разница между хаотичность родитеьской вершины и дочерних вершин, тем лучше данный предикат.

**Нормировка**

Нормировка нужна для того, чтобы не было выгодно разделять по одному объекту. Например, имеется 5 крестиков и 5 ноликов, тогда влевую часть можно отправить один нолик, а вправую 5 крестиков и 4 нолика. Это происходит, так как у данного разбиения в левой ветке критерий информативности будет равен 0. Если сравнить такое разбиение и разбиение где объектов одинаковое количество, тогда может оказаться, что текущее разбиение может оказаться лучше с точки зрения критерия $Q$, поэтому мы вводим эти ввеса для того, чтобы вершина с небольшим количеством объектов не вносила большого вклада в функционал. Тем самым не будет происходить разбиений где выбирается только один объект.


**Зачем $R_m$ в $Q(R_m, j, s)$?**

Так как $H(R_m)$ константа, зачем он в функционале ошибки если его можна убать? Если использовать в функционале $H(R_m)$, тогда это дает некоторый смысл "Насколько уменьшилось impurity за счет данного разбиения".

**Вид формулы:**

$H(R_m)$ можно не записывать в этой задаче и тогда перепишем в формул в следующем виде, которую будем минимизировать:

$$\large Q(R_m, j, s)
=
\frac{|R_\ell|}{|R_m|}
H(R_\ell)
+
\frac{|R_r|}{|R_m|}
H(R_r).$$


Здесь $H(R)$ - *это критерий информативности (impurity criterion)*, который оценивает качество распределения целевой переменной среди объектов множества $R$. Чем меньше разнообразие целевой переменной, тем меньше должно быть значение критерия информативности - и, соответственно, мы будем пытаться минимизировать его значение. Функционал качества $Q(R_m, j, s)$ мы при этом будем максимизировать.

<h1 style="color:#008B8B">6. Обработка пропущенных значений</h1>

### Способ 1

**Решение проблемы выбора предиката**

При выборе лучшего предиката $[x_j \le t]$ в некоторой вершине происходит перебор по всем $j, t$ и для некоторых объетов мы не знаем $x_j$, значит мы не знаем куда относить этот объект влево или в право, следовательно мы не сможем посчитать критерий качества предиката.

* Считаем качество $Q(R_m, j, s)$ только по объектам, для которых знаем значение $x_j$

* После выборка предиката мы отправляем объекты влевую часть и правую

**Решение проблемы применения дерева к объекту с пропуском**

Если имеется предикат $[x_j \le t]$ и мы не знаем значение $x_j$ признака объекта, тогда если мы сейчас в вершине $m$, тогда прогноз в этой вершине считается как усредненный прогноз левого поддерева и правого поддерева:

$\large a_m(x) = \frac{|R_\ell|}{|R_m|} a_{\ell}(x) + \frac{|R_r|}{|R_m|} a_{r}(x)$

В случае задачии классификации выдаются вероятности классов, тогда мы будем усреднять вероятности классов из левого и правого поддерева, можно ещё применять нормировку, так как они будут суммироваться не в 1.

### Способ 2 - применение суррогатных предикатов

Для вершины m имеется предикат $[x_j \le t]$. Найдем предика с другим признаком $[x_k \le \tilde{t}]$, который максимально похожее разбиение. Такой предикат, у которого объекты попадающие влево и вправо как можно сильнее пересекаются с объектами предиката $[x_j \le t]$. Этот новый предикат будет называться сурагатным и на этапе. Когда мы будем применять объект для дерева и если для предиката $[x_j \le t]$ мы не знаем значение $x_j$, тогда для принятия решения, куда необходимо отправить объект будем использовать сурагатный предикат.

<h1 style="color:#008B8B">7. Учет категориальных признаков</h1>

### Способ 1 (multi-way splits)

Если имеются категориальные признаки, тогда будем делать вершины у которых не два потомка, а потомков столько, сколько значений у категориального признака.

**Как выбрать предикат?**

В этом случае качество будем считать по следующей формуле:

$$\large Q(R_m, j, s)
=
H(R_m)
-
\frac{|R_1|}{|R_m|}
H(R_1)
- \ldots -
\frac{|R_c|}{|R_m|}
H(R_c).$$

Этот подход позволят отчень быстро переобучиться, так как для дерева глубины 1 мы получим столько листьев, сколько категорий. 

Если мы разбиваем по числовому признаку, тогда разбиение происходит только на два поддерева, а по категориальному мы разбиваем на столько поддеревьем, сколько у нас категорий. И понятно, что качество для категориального разбиения с большим количеством деревьев будет выше, значит мы не сможим выбрать между числовми и категориальными признаками.

### Способ 2

Пусть категориальный признак $x_j$ имеет множество значений $Q = \{u_1, \dots, u_q\}$, $|Q| = q$. Разобьем множество значений на два непересекающихся подмножества: $Q = Q_1 \sqcup Q_2$, и определим предикат как индикатор попадания в первое подмножество: $\beta(x) = [x_j \in Q_1]$. Таким образом, объект будет попадать в левое поддерево, если признак $x_j$ попадает в множество $Q_1$, и в первое поддерево в противном случае. Основная проблема заключается в том, что для построения оптимального предиката нужно перебрать $2^{q - 1} - 1$ вариантов разбиения, что может быть не вполне возможным.


Оказывается, можно обойтись без полного перебора в случаях с бинарной
классификацией и регрессией. Обозначим через $R_m(u)$ множество объектов, которые попали в вершину $m$ и у которых $j$-й признак имеет значение $u$; через $N_m(u)$ обозначим количество таких объектов.

В случае с бинарной классификацией упорядочим все значения категориального признака на основе того, какая доля объектов с таким значением имеет класс $+1$:

$$\frac{1}{N_m(u_{(1)})}
\sum_{x_i \in R_m(u_{(1)})}
    [y_i = +1]
\leq
\dots
\leq
\frac{1}{N_m(u_{(q)})}
\sum_{x_i \in R_m(u_{(q)})}
    [y_i = +1],$$

после чего заменим категорию $u_{(i)}$ на число $i$, и будем искать разбиение как для вещественного признака. Можно показать, что если искать оптимальное разбиение по критерию Джини или энтропийному критерию, то мы получим такое же разбиение, как и при переборе по всем возможным $2^{q - 1} - 1$ вариантам.

Для задачи регрессии с MSE-функционалом это тоже будет верно, если упорядочивать значения признака по среднему ответу объектов с таким значением:

$$\frac{1}{N_m(u_{(1)})}
\sum_{x_i \in R_m(u_{(1)})}
    y_i
\leq
\dots
\leq
\frac{1}{N_m(u_{(q)})}
\sum_{x_i \in R_m(u_{(q)})}
    y_i.$$
    
Именно такой подход используется в библиотеке
Spark MLlib http://spark.apache.org/docs/latest/mllib-decision-tree.html

<h1 style="color:#008B8B">9. Решающие деревья и линейные модели</h1> 

Как следует из определения, решающее дерево $a(x)$ разбивает всё признаковое пространство на некоторое количество непересекающихся подмножеств $\{J_1, \dots, J_n\}$, и в каждом подмножестве $J_j$ выдаёт константный прогноз $w_j$. Значит, соответствующий алгоритм можно записать аналитически:

$a(x)
=
\sum\limits_{j = 1}^{n}
    w_j
    [x \in J_j].$

Обратим внимание, что это линейная модель с весами $w_1, \ldots, w_n$ над признаками $([x \in J_j])_{j = 1}^{n}$. Дерево разбивает признаковое пространство на области выдавая новые нелинейные признаки, которые являются индикаторами отношения объекта к некой области и над этими признаками строит линейную модель. Получается, что решающее дерево с помощью жадного алгоритма подбирает преобразование признаков для данной задачи, а затем просто строит линейную модель над этими признаками. Далее мы увидим, что многие нелинейные методы машинного обучения можно представить как совокупность линейных методов и хитрых способов порождения признаков. 

Можем например, строить дерево, берем индикаторы поподания в листья, добавляем ещё признаков и строим над этим линейную модель. Можно зафиксировать эти индикаторы $[x \in J_j]$ и очень легко перестраивать линейную модель для новых данных, тем самым мы быстро будем обновлять веса, что быстрее чем каждый раз строить новое дерево.

**Случайные предикаты**

Деревья можно использовать как генераторы новых признаков. Берем некоторое дерево, где предикаты в каждой вершине выбираются случайно. После из этого дерева извлекается категориальный признак - номер листа, в который попал объект. Другими словами, все области, которые были получины случаным разбиением предиката необходимо пронумеровать и после каждому объекту сопоставляем в какой области признакового пространства содержится объект. После мы полуим новый нелинейный признак, который можно использовать в линейной модели. 

Это позволяет сократить скорость обучения дерева, так как в каждой вершине необходимо перебрать все возможные признаки и пороги.